In [1]:
#IMPORTING THE LIBRARIES
import random as rd
import numpy as np 
import matplotlib.pyplot as plt 
import statistics as stat
import math
import pandas as pd
from scipy import integrate as intg
from scipy.stats import linregress
import warnings
import scipy.stats as si
np.seterr(divide='ignore', invalid='ignore')
import scipy.integrate as integrate
import cmath
import scipy
warnings.filterwarnings('ignore')

import scipy.stats as st
import scipy.fft as fft
import scipy.interpolate as interpolate
import statsmodels.api as sm
from tqdm import tqdm
import time # to obtain the running time of some functions
from toolkit import generic_CF, genericFFT, odeRK4

## model

### Shot noise

In [2]:
## solve for ChF of our proposed dynamics
def coef_A(tau, u, *var):
    '''
    Given u, compute A(tau,u) as a function of tau
    For our model, the coef for X0 (vix0S)
    '''
    kappa = var[0]
    return 1j * u * np.exp(-kappa*tau)


def coef_E(u, *var):
    '''
    Given u, compute E(tau,u) as a function of tau
    For our model, the coef for m0
    Output:
        E_tau_u_func: function of time t
    '''
    T, kappa, kappam, omegam = var
    E_0_u = 0
    nU = len(u)
    
    def RHS_E(t,y):
        tmp1 = kappa*coef_A(t,u, kappa)
        tmp2 = -kappam*y
        # tmp3 = 0.5*omegam**2*y**2, # CIR
        tmp3 = 0, # OU
        return tmp1+tmp2+tmp3

    time, E_tau_u = odeRK4(RHS_E, 0, T, E_0_u, 100, nU)
    E_tau_u_func = interpolate.interp1d(time, E_tau_u, kind='cubic')
    
    return E_tau_u_func


def coef_B(u, *var):
    '''
    Given u, compute B(tau,u) as a function of tau
    For our model, the coef for v0
    Output:
        B_tau_u_func: function of time t
    '''
    T, kappa, kappai, omegai, rhoi = var
    B_0_u = 0
    nU = len(u)
    
    def RHS_B(t,y):
        tmp1 = -kappai*y
        tmp2 = 0.5*coef_A(t,u, kappa)**2
        tmp3 = 0.5*omegai**2*y**2
        tmp4 = rhoi*omegai*coef_A(t,u, kappa)*y
        return tmp1+tmp2+tmp3+tmp4

    time, B_tau_u = odeRK4(RHS_B, 0, T, B_0_u, 100, nU)
    B_tau_u_func = interpolate.interp1d(time, B_tau_u, kind='cubic')
    
    return B_tau_u_func


def coef_C(u, *var):
    '''
    Given u, compute C(tau,u) as a function of tau
    For our model, the coef for L^s_0
    Output:
        C_tau_u_func: function of time t
    '''
    T, kappa, bs = var
    C_0_u = 0
    nU = len(u)
    
    def RHS_C(t,y):
        tmp1 = -bs*coef_A(t,u, kappa)
        tmp2 = -bs*y
        return tmp1+tmp2

    time, C_tau_u = odeRK4(RHS_C, 0, T, C_0_u, 100, nU)
    C_tau_u_func = interpolate.interp1d(time, C_tau_u, kind='cubic')
    
    return C_tau_u_func


def coef_D(u, *var):
    '''
    Given u, compute D(tau,u) as a function of tau
    For our model, the coef for L^v_0
    Output:
        D_tau_u_func: function of time t
    '''
    T, kappa, kappa1, omega1, rho1, bv = var
    D_0_u = 0
    nU = len(u)
    
    coef_B1_func_ = coef_B(u, T, kappa, kappa1, omega1, rho1)
    
    def RHS_D(t,y):
        tmp1 = -bv*coef_B1_func_(t)
        tmp2 = -bv*y
        return tmp1+tmp2

    time, D_tau_u = odeRK4(RHS_D, 0, T, D_0_u, 100, nU)
    D_tau_u_func = interpolate.interp1d(time, D_tau_u, kind='cubic')
    
    return D_tau_u_func


def coef_F(u, *var):
    '''
    Given u, compute F(tau,u) as a function of tau
    For our model, the const term
    '''
    T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv, lamb, muJS, muJV = var
    F_0_u = 0
    nU = len(u)
    
    coef_B_func_ = coef_B(u, T, kappa, kappa1, omega1, rho1)
    coef_C_func_ = coef_C(u, T, kappa, bs)
    coef_D_func_ = coef_D(u, T, kappa, kappa1, omega1, rho1, bv)
    coef_E_func_ = coef_E(u, T, kappa, kappam, omegam)
    
    def RHS_F(t,y):
        tmp1 = kappa1*theta1*coef_B_func_(t) ## B for v
        tmp2 = kappam*thetam*coef_E_func_(t)  ## E for m
        tmp2 = tmp2 + 0.5*omegam**2*coef_E_func_(t)**2 ## m, additional adjustment for OU
        ## for jump
        jump_s = 1/(1 - (coef_A(t,u, kappa)+coef_C_func_(t)) * muJS)
        jump_v = 1/(1 - (coef_B_func_(t)+coef_D_func_(t)) * muJV)
        tmp3 = lamb * (jump_s*jump_v - 1) 
        return tmp1+tmp2+tmp3

    time, F_tau_u = odeRK4(RHS_F, 0, T, F_0_u, 100, nU)
    F_tau_u_func = interpolate.interp1d(time, F_tau_u, kind='cubic')
    
    return F_tau_u_func


def ChF_Our(u,*var):
    '''
    ChF of log price XT under our model
    Parameters:
    params: model specific parameters, see Ye's research note
    ini_set: initial value for VIX, v1, L^s,L^v, m
    '''
    
    params = var[0]
    T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv, lamb, muJS, muJV = params
    ini_set = var[1]
    VIX0, v10, Ls0,Lv0, m0 = ini_set
    vix0 = np.log(VIX0) ## as the model is for vix = log(VIX)
    
    coef_B_func_ = coef_B(u, T, kappa, kappa1, omega1, rho1)
    coef_C_func_ = coef_C(u, T, kappa, bs)
    coef_D_func_ = coef_D(u, T, kappa, kappa1, omega1, rho1, bv)
    coef_E_func_ = coef_E(u, T, kappa, kappam, omegam)
    coef_F_func_ = coef_F(u, T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv, lamb, muJS, muJV)
    
    res = np.exp(coef_A(T,u, kappa)*vix0+\
                 coef_B_func_(T)*v10+\
                 coef_C_func_(T)*Ls0+\
                 coef_D_func_(T)*Lv0+\
                 coef_E_func_(T)*m0+\
                 coef_F_func_(T))
    return res

## <a id='toc1_1_'></a>[Pricing via COS](#toc0_)

如果要写Lewis的话，最后的数值积分应该write by hand

In [3]:
def CallPutCoefficients(CP,a,b,k):
    """ 
    Determine coefficients for Call or Put Prices 
    """
    if str(CP).lower()=="c" or str(CP).lower()=="1":                  
        c = 0.0
        d = b
        coef = Chi_Psi(a,b,c,d,k)
        Chi_k = coef["chi"]
        Psi_k = coef["psi"]
        if a < b and b < 0.0:
            H_k = np.zeros([len(k),1])
        else:
            H_k      = 2.0 / (b - a) * (Chi_k - Psi_k)  
        
    elif str(CP).lower()=="p" or str(CP).lower()=="-1":
        c = a
        d = 0.0
        coef = Chi_Psi(a,b,c,d,k)
        Chi_k = coef["chi"]
        Psi_k = coef["psi"]
        H_k      = 2.0 / (b - a) * (- Chi_k + Psi_k)               
    
    return H_k  


def Chi_Psi(a,b,c,d,k):
    psi = np.sin(k * np.pi * (d - a) / (b - a)) - np.sin(k * np.pi * (c - a)/(b - a))
    psi[1:] = psi[1:] * (b - a) / (k[1:] * np.pi)
    psi[0] = d - c
    
    chi = 1.0 / (1.0 + np.power((k * np.pi / (b - a)) , 2.0)) 
    expr1 = np.cos(k * np.pi * (d - a)/(b - a)) * np.exp(d)  - np.cos(k * np.pi 
                  * (c - a) / (b - a)) * np.exp(c)
    expr2 = k * np.pi / (b - a) * np.sin(k * np.pi * 
                        (d - a) / (b - a))   - k * np.pi / (b - a) * np.sin(k 
                        * np.pi * (c - a) / (b - a)) * np.exp(c)
    chi = chi * (expr1 + expr2)
    
    value = {"chi":chi,"psi":psi }
    return value


## Fourier COS method for Option Pricing
## can simutaneously process a set of strike prices
def genericCOS(r,tau,params,ini_set,CP,K,N,L):
    # cf      - characteristic function as a functon
    # CP      - C for call and P for put
    # ini_set - Initial stock price, initial v0, etc
    # N       - Number of expansion terms
    # L       - size of truncation domain (typ.:L=8 or L=10)  
    # K       - can be a vector, strike price
        
    # reshape K to a column vector
    if isinstance(K, list):
        K = K
    else:
        K = [K]
    K = np.array(K).reshape([len(K),1])
    
    # assigning i=sqrt(-1)
    i = 1j 
    
    S0 = ini_set[0]
    x0 = np.log(S0 / K)   
    
    # truncation domain
    a = 0.0 - L * np.sqrt(tau)
    b = 0.0 + L * np.sqrt(tau)

    # sumation from k = 0 to k=N-1
    k = np.linspace(0,N-1,N).reshape([N,1])  
    u = k * np.pi / (b - a);
    ## reshape u to be (128, ) so that our ChF can process
    u = u.reshape([-1])

    # Determine coefficients for Put Prices  
    H_k = CallPutCoefficients(CP,a,b,k)
       
    mat = np.exp(i * np.outer((x0 - a) , u))

    cf = lambda u: ChF_Our(u, params,ini_set) * np.exp(-1j*np.log(S0)*u) 
    ## reshape cf(u) to be (128, 1)
    cf_u = cf(u)
    temp = cf_u.reshape([-1,1]) * H_k
    # temp = cf(u) * H_k
    temp[0] = 0.5 * temp[0]    
    
    ## now value is a column vector
    value = np.exp(-r * tau) * K * np.real(mat.dot(temp))
         
    return value.reshape([-1])

In [4]:
%%time
r,K, T, kappa = 0.03,40, 90/360, 5
kappam, thetam, omegam = 2.0, 3.0, 0.25
kappa1, theta1, omega1, rho1 = 2.0, 2.0, 3.0, 0.8 
bs,bv = 0.2,0.05
lamb, muJS, muJV = 1.5, 0.2, 0.2
params = [T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv, lamb, muJS, muJV]

VIX_0, v10, Ls0,Lv0, m0 = 40, 0.64, 0.,0., 3
ini_set = [VIX_0, v10, Ls0,Lv0, m0]
CP = "c"

## COS setting
N = 2**7
L = 10

call_cos = genericCOS(r,T,params,ini_set,CP,K,N,L)
np.round(call_cos[0],4)

CPU times: user 332 ms, sys: 3.63 ms, total: 335 ms
Wall time: 328 ms


1.5381

In [10]:
%%time
r,K, T, kappa = 0.03,40, 90/360, 5
kappam, thetam, omegam = 2.0, 3.0, 0.25
kappa1, theta1, omega1, rho1 = 2.0, 2.0, 3.0, 0.8 
bs,bv = 300,0.05
lamb, muJS, muJV = 1.5, 0.2, 0.2
params = [T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv, lamb, muJS, muJV]

VIX_0, v10, Ls0,Lv0, m0 = 40, 0.64, 0.,0., 3
ini_set = [VIX_0, v10, Ls0,Lv0, m0]
CP = "c"

## COS setting
N = 2**7
L = 10

call_cos = genericCOS(r,T,params,ini_set,CP,K,N,L)
np.round(call_cos[0],4)

CPU times: user 308 ms, sys: 0 ns, total: 308 ms
Wall time: 302 ms


1.176

### <a id='toc1_1_1_'></a>[encapsule](#toc0_)

In [5]:
def Call_Pricer_COS(r, T, K, params, ini_set):
    '''
    params: T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv, lamb, muJS, muJV
    ini_set: initial value of state variables
    '''
    ## COS setting
    N, L = 2**7, 10
    call_cos = genericCOS(r,T,params,ini_set,CP,K,N,L)
    return np.round(call_cos[0], 4)

In [6]:
Call_Pricer_COS(r, T, 35, params, ini_set)

2.1304

In [7]:
Call_Pricer_COS(r, T, 40, params, ini_set)

1.5381

In [13]:
K_ = np.linspace(30, 80, 11)
for i in tqdm(range(len(K_))):
    price_ = Call_Pricer_COS(r, T, K_[i], params, ini_set)
    print("Strike: "+str(K_[i]), "call price: ", str(price_))

  9%|▉         | 1/11 [00:00<00:02,  4.32it/s]

Strike: 30.0 call price:  3.0784


 18%|█▊        | 2/11 [00:00<00:02,  4.29it/s]

Strike: 35.0 call price:  2.1304


 27%|██▋       | 3/11 [00:00<00:01,  4.19it/s]

...

Strike: 70.0 call price:  0.3723


 91%|█████████ | 10/11 [00:02<00:00,  4.25it/s]

Strike: 75.0 call price:  0.3113


100%|██████████| 11/11 [00:02<00:00,  4.25it/s]

Strike: 80.0 call price:  0.2627


## <a id='toc1_2_'></a>[Pricing via MC](#toc0_)

In [14]:
def GeneratePathsEuler(NoOfPaths,NoOfSteps, params, ini_set, jump_param):
    '''
    Monte Carlo Simulation for our model as sanity check for FFT pricing method
    simulate vix_t = log VIX_t
    kappa: speed of mean-reverting
    gamma: vol of vol
    rho: correlation between 2 BMs
    theta: long-term variance
    '''
    T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv = params
    S_0, v10, Ls0,Lv0, m0 = ini_set
    muJ, eta, muJv = jump_param
    
    time = np.zeros([NoOfSteps+1])
    dt = T / float(NoOfSteps)

    ## diffusion terms
    B  = np.random.normal(0.0,1.0,[NoOfPaths,NoOfSteps])  # BM for central tendency m
    Z1 = np.random.normal(0.0,1.0,[NoOfPaths,NoOfSteps]) # BM for vix (log VIX)
    Z3 = np.random.normal(0.0,1.0,[NoOfPaths,NoOfSteps]) # uncorrelated BM for v1
    Z_3 = np.zeros_like(Z3) ## correlated BM for v1
    
    ## jump terms
    ZPois = np.random.poisson(eta*dt,[NoOfPaths,NoOfSteps])    
    J = np.random.exponential(muJ,  [NoOfPaths,NoOfSteps])
    Jv= np.random.exponential(muJv,  [NoOfPaths,NoOfSteps])
    
    ## initiate paths for 4 processes
    V1 = np.zeros([NoOfPaths, NoOfSteps+1])
    m = np.zeros([NoOfPaths, NoOfSteps+1])
    X = np.zeros([NoOfPaths, NoOfSteps+1])
    Ls = np.zeros([NoOfPaths, NoOfSteps+1]) ## correlated BM for L^s
    Lv = np.zeros([NoOfPaths, NoOfSteps+1]) ## correlated BM for L^v
    
    V1[:,0] = v10
    m[:,0] = m0
    X[:,0] = np.log(S_0)
    Ls[:,0] = Ls0
    Lv[:,0] = Lv0
    
    for i in range(0, NoOfSteps):
        Z_3[:,i] = rho1 * Z1[:,i] + np.sqrt(1.0-rho1**2)*Z3[:,i]
        
        # shot for variance
        Lv[:,i+1] = Lv[:,i] - bv*Lv[:,i] * dt + Jv[:,i]*ZPois[:,i]
        
        # Truncated boundary condition
        m[:, i+1] = m[:, i] + kappam*(thetam - m[:, i]) * dt + omegam * dt**0.5*B[:, i]
        m[:, i+1] = np.maximum(m[:, i+1], 1e-6)
        V1[:,i+1] = V1[:,i] + kappa1*(theta1 - V1[:,i]) * dt + omega1* np.sqrt(V1[:,i]) * dt**0.5*Z_3[:,i] + (Lv[:,i+1] - Lv[:,i])
        V1[:,i+1] = np.maximum(V1[:,i+1], 1e-6)
        
        # shot noise for log VIX
        Ls[:,i+1] = Ls[:,i] - bs*Ls[:,i] * dt + J[:, i]*ZPois[:,i]
        
        ## inclusion of co-Jumps
        X[:,i+1] = X[:, i] + kappa *(m[:, i] - X[:, i]) * dt + np.sqrt(V1[:,i]) * dt**0.5*Z1[:,i] + (Ls[:,i+1] - Ls[:,i])
        time[i+1] = time[i] + dt

    # Compute exponent
    S = np.exp(X)
    return time, S, Ls, Lv

In [15]:
%%time
## set random seed for reproducibility
np.random.seed(10)

## set parameters for numerical illustration
r,K, T, kappa = 0.03,40, 90/360, 5
kappam, thetam, omegam = 2.0, 3.0, 0.25
kappa1, theta1, omega1, rho1 = 2.0, 2.0, 3.0, 0.8 
bs,bv = 0.2, 0.05
params = [T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv]

VIX_0, v10, Ls0,Lv0, m0 = 40, 0.64, 0.,0., 3
ini_set = [VIX_0, v10, Ls0,Lv0, m0]

muJS,eta,muJv = 0.2, 1.5, 0.2
jump_param = [muJS,eta,muJv]

NoOfSteps, NoOfPaths = 256, 100000 ## number of time steps and paths

## main test
t, VIX, Ls,Lv = GeneratePathsEuler(NoOfPaths,NoOfSteps, params, ini_set, jump_param)
call_mc = np.exp(-r*T)*np.mean(np.maximum(0, VIX[:,-1]-K))
print("Option via Monte Carlo: for strike %s the option premium is %6.4f" % (K, call_mc))

Option via Monte Carlo: for strike 40 the option premium is 1.5357
CPU times: user 12.2 s, sys: 432 ms, total: 12.7 s
Wall time: 12.7 s


### <a id='toc1_2_1_'></a>[encapsule](#toc0_)

In [21]:
def Call_Pricer_MC(r, T, K, params, ini_set, jump_param):
    NoOfSteps, NoOfPaths = 256, 500000 ## number of time steps and paths
    t, VIX,Ls,Lv = GeneratePathsEuler(NoOfPaths,NoOfSteps, params, ini_set, jump_param)
    call_mc = np.exp(-r*T)*np.mean(np.maximum(0, VIX[:,-1]-K))
    return call_mc

In [122]:
## set parameters for numerical illustration
r, T, kappa = 0.03, 90/360, 5
kappam, thetam, omegam = 2.0, 3.0, 0.25
kappa1, theta1, omega1, rho1 = 2.0, 2.0, 3.0, 0.8 
bs,bv = 0.2,0.05
params = [T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv]

## ini_set
VIX_0, v10, Ls0,Lv0, m0 = 40, 0.64, 0.,0., 3
ini_set = [VIX_0, v10, Ls0,Lv0, m0]

## jump_param
muJS,eta,muJv = 0.2, 1.5, 0.2
jump_param = [muJS,eta,muJv]

### <a id='toc1_2_2_'></a>[testing case](#toc0_)

In [ ]:
K_ = np.linspace(30,80,11)
call_ = np.zeros_like(K_)
for i in tqdm(range(len(call_))):
    call_[i] = Call_Pricer_MC(r, T, K_[i], params, ini_set, jump_param)
    print(call_[i])

  0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
K = 40
call_ = np.zeros(10)
for i in tqdm(range(len(call_))):
    call_[i] = Call_Pricer_MC(r, T, K, params, ini_set, jump_param)
np.mean(call_)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
K = 35
call_ = np.zeros(10)
for i in tqdm(range(len(call_))):
    call_[i] = Call_Pricer_MC(r, T, K, params, ini_set, jump_param)
np.mean(call_)

In [ ]:
K = 45
call_ = np.zeros(10)
for i in tqdm(range(len(call_))):
    call_[i] = Call_Pricer_MC(r, T, K, params, ini_set, jump_param)
np.mean(call_)

## INACTIVE, ChF, m with CIR

In [ ]:
## solve for ChF of our proposed dynamics
def coef_A(tau, u, *var):
    '''
    Given u, compute A(tau,u) as a function of tau
    For our model, the coef for X0 (vix0S)
    '''
    kappa = var[0]
    return 1j * u * np.exp(-kappa*tau)


def coef_E(u, *var):
    '''
    Given u, compute E(tau,u) as a function of tau
    For our model, the coef for m0
    Output:
        E_tau_u_func: function of time t
    '''
    T, kappa, kappam, omegam = var
    E_0_u = 0
    nU = len(u)
    
    def RHS_E(t,y):
        tmp1 = kappa*coef_A(t,u, kappa)
        tmp2 = -kappam*y
        tmp3 = 0.5*omegam**2*y**2
        return tmp1+tmp2+tmp3

    time, E_tau_u = odeRK4(RHS_E, 0, T, E_0_u, 100, nU)
    E_tau_u_func = interpolate.interp1d(time, E_tau_u, kind='cubic')
    
    return E_tau_u_func


def coef_B(u, *var):
    '''
    Given u, compute B(tau,u) as a function of tau
    For our model, the coef for v0
    Output:
        B_tau_u_func: function of time t
    '''
    T, kappa, kappai, omegai, rhoi = var
    B_0_u = 0
    nU = len(u)
    
    def RHS_B(t,y):
        tmp1 = -kappai*y
        tmp2 = 0.5*coef_A(t,u, kappa)**2
        tmp3 = 0.5*omegai**2*y**2
        tmp4 = rhoi*omegai*coef_A(t,u, kappa)*y
        return tmp1+tmp2+tmp3+tmp4

    time, B_tau_u = odeRK4(RHS_B, 0, T, B_0_u, 100, nU)
    B_tau_u_func = interpolate.interp1d(time, B_tau_u, kind='cubic')
    
    return B_tau_u_func


def coef_C(u, *var):
    '''
    Given u, compute C(tau,u) as a function of tau
    For our model, the coef for L^s_0
    Output:
        C_tau_u_func: function of time t
    '''
    T, kappa, bs = var
    C_0_u = 0
    nU = len(u)
    
    def RHS_C(t,y):
        tmp1 = -bs*coef_A(t,u, kappa)
        tmp2 = -bs*y
        return tmp1+tmp2

    time, C_tau_u = odeRK4(RHS_C, 0, T, C_0_u, 100, nU)
    C_tau_u_func = interpolate.interp1d(time, C_tau_u, kind='cubic')
    
    return C_tau_u_func


def coef_D(u, *var):
    '''
    Given u, compute D(tau,u) as a function of tau
    For our model, the coef for L^v_0
    Output:
        D_tau_u_func: function of time t
    '''
    T, kappa, kappa1, omega1, rho1, bv = var
    D_0_u = 0
    nU = len(u)
    
    coef_B1_func_ = coef_B(u, T, kappa, kappa1, omega1, rho1)
    
    def RHS_D(t,y):
        tmp1 = -bv*coef_B1_func_(t)
        tmp2 = -bv*y
        return tmp1+tmp2

    time, D_tau_u = odeRK4(RHS_D, 0, T, D_0_u, 100, nU)
    D_tau_u_func = interpolate.interp1d(time, D_tau_u, kind='cubic')
    
    return D_tau_u_func


def coef_F(u, *var):
    '''
    Given u, compute F(tau,u) as a function of tau
    For our model, the const term
    '''
    T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv, lamb, muJS, muJV = var
    F_0_u = 0
    nU = len(u)
    
    coef_B_func_ = coef_B(u, T, kappa, kappa1, omega1, rho1)
    coef_C_func_ = coef_C(u, T, kappa, bs)
    coef_D_func_ = coef_D(u, T, kappa, kappa1, omega1, rho1, bv)
    coef_E_func_ = coef_E(u, T, kappa, kappam, omegam)
    
    def RHS_F(t,y):
        tmp1 = kappa1*theta1*coef_B_func_(t) ## B for v
        tmp2 = kappam*thetam*coef_E_func_(t)  ## E for m
        ## for jump
        jump_s = 1/(1 - (coef_A(t,u, kappa)+coef_C_func_(t)) * muJS)
        jump_v = 1/(1 - (coef_B_func_(t)+coef_D_func_(t)) * muJV)
        tmp3 = lamb * (jump_s*jump_v - 1) 
        return tmp1+tmp2+tmp3

    time, F_tau_u = odeRK4(RHS_F, 0, T, F_0_u, 100, nU)
    F_tau_u_func = interpolate.interp1d(time, F_tau_u, kind='cubic')
    
    return F_tau_u_func


def ChF_Our(u,*var):
    '''
    ChF of log price XT under our model
    Parameters:
    params: model specific parameters, see Ye's research note
    ini_set: initial value for VIX, v1, L^s,L^v, m
    '''
    
    params = var[0]
    T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv, lamb, muJS, muJV = params
    ini_set = var[1]
    VIX0, v10, Ls0,Lv0, m0 = ini_set
    vix0 = np.log(VIX0) ## as the model is for vix = log(VIX)
    
    coef_B_func_ = coef_B(u, T, kappa, kappa1, omega1, rho1)
    coef_C_func_ = coef_C(u, T, kappa, bs)
    coef_D_func_ = coef_D(u, T, kappa, kappa1, omega1, rho1, bv)
    coef_E_func_ = coef_E(u, T, kappa, kappam, omegam)
    coef_F_func_  = coef_F(u, T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv, lamb, muJS, muJV)
    
    res = np.exp(coef_A(T,u, kappa)*vix0+\
                 coef_B_func_(T)*v10+\
                 coef_C_func_(T)*Ls0+\
                 coef_D_func_(T)*Lv0+\
                 coef_E_func_(T)*m0+\
                 coef_F_func_(T))
    return res